In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [162]:
import pandas as pd
import numpy as np
import tensorflow
import requests
from bs4 import BeautifulSoup

In [163]:
response=requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')
soup=BeautifulSoup(response.content,'html.parser')


## Data Cleaning

In [ ]:
import spacy
import re,string
from nltk.corpus import stopwords
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 5281520

def preprocess(texts):
    doc = nlp(texts)
    text=" " .join([token.lemma_ for token in doc])
    text=' '.join(word for word in text.split() if word.lower() not in stopwords.words('english'))
    text = re.sub(r'\&\w*;','',text)
    text = re.sub('@[^\s]+','',text)
    text = re.sub(r'\$\w*','',text)
    text = text.lower()
    text = re.sub(r'https?:\/\/.*\/\w*','',text)
    text = re.sub(r'#\w*','',text)
    text = re.sub(r'[' + string.punctuation.replace('@','') + ']+',' ',text)
    text = re.sub(r'\b\w{1,2}\b','',text)
    text = re.sub(r'\s\s+',' ',text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text=re.sub(r'\s+', ' ', text)
    text = text.lstrip(' ')
    text = ''.join(c for c in text if c <= '\uFFFF')
    return text


In [177]:
data=preprocess(soup.get_text())

In [178]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
vocab_size=len(tokenizer.word_index)+1

tokenizer=Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data)
sequence=tokenizer.texts_to_sequences(data)



In [179]:
sequences_padded = pad_sequences(sequence,maxlen=30, padding='pre')
sequences= np.array(sequences_padded)
x=sequences[:,:-1]
yx=sequences[:,-1]
batch_size=64
y=to_categorical(yx,num_classes=vocab_size)

In [142]:
import numpy as np

# Example sequences
sequences = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Separate input sequences (x) and target sequences (y)
x = sequences[:, :-1]
y = sequences[:, -1]

print("Input Sequences (x):")
print(x)
print("\nTarget Sequences (y):")
print(y)

Input Sequences (x):
[[1 2]
 [4 5]
 [7 8]]

Target Sequences (y):
[3 6 9]


In [180]:
print("Shape of x:", x.shape)
seq_length= x.shape[1]

Shape of x: (2834840, 29)


In [182]:
from keras.layers import LSTM,Dense,Embedding,Dropout
from keras.models import Sequential
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length)) 
model.add(LSTM(units=32,return_sequences=True))

model.add(LSTM(units=16))
model.add(Dropout(0.3))

model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))

model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 29, 50)            1900      
                                                                 
 lstm_38 (LSTM)              (None, 29, 32)            10624     
                                                                 
 lstm_39 (LSTM)              (None, 16)                3136      
                                                                 
 dropout_17 (Dropout)        (None, 16)                0         
                                                                 
 dense_51 (Dense)            (None, 16)                272       
                                                                 
 dense_52 (Dense)            (None, 16)                272       
                                                                 
 dense_53 (Dense)            (None, 38)              

In [183]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(
    x,
    y,
    batch_size=batch_size,
    epochs=3
)

Epoch 1/3
44295/44295 [==============================] - 317s 7ms/step - loss: 2.9041 - accuracy: 0.1599
Epoch 2/3
44295/44295 [==============================] - 312s 7ms/step - loss: 2.9019 - accuracy: 0.1600
Epoch 3/3
44295/44295 [==============================] - 311s 7ms/step - loss: 2.9018 - accuracy: 0.1600


In [192]:
def generate_text(seed_text, model, tokenizer, max_sequence_length, num_words_to_generate):
    generated_text = seed_text  # Initialize with the seed text
    for _ in range(num_words_to_generate):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        while len(token_list) < max_sequence_length - 1:
            token_list.insert(0, 0)  
        token_list = token_list[-(max_sequence_length-1):]
        input_sequence = np.array(token_list)
        input_sequence = np.reshape(input_sequence, (1, -1))
        predicted_probs = model.predict(input_sequence, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, '')
        seed_text += ' ' + predicted_word
        generated_text += ' ' + predicted_word

    return generated_text

# Example Usage
generated_text = generate_text(seed_text='''hello ''', model=model, tokenizer=tokenizer,
                               max_sequence_length=seq_length+1, num_words_to_generate=10)
print("Generated Text:", generated_text)


Generated Text: hello           


In [197]:
def generate_text(seed_text, model, tokenizer, max_sequence_length, num_words_to_generate):
    generated_text = seed_text  # Initialize with the seed text
    for _ in range(num_words_to_generate):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        while len(token_list) < max_sequence_length - 1:
            token_list.insert(0, 0)  
        token_list = token_list[-(max_sequence_length-1):]
        input_sequence = np.array(token_list)
        input_sequence = np.reshape(input_sequence, (1, -1))
        predicted_probs = model.predict(input_sequence, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, '')
        seed_text += ' ' + predicted_word
        generated_text += ' ' + predicted_word

    return generated_text

# Example Usage
generated_text = generate_text(seed_text='''The Bird sang''', model=model, tokenizer=tokenizer,
                               max_sequence_length=seq_length+1, num_words_to_generate=123)
print("Generated Text:", generated_text)
print("Generated Text:", len(generated_text))



Generated Text: The Bird sang                                                                                                                           
Generated Text: 136
